In [42]:
import requests
from urllib.parse import urlparse, parse_qsl, urlencode

In [43]:
GOOGLE_API_KEY = '******'

Google Map Client API

In [44]:
class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key is required")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299): 
            return {}
        latlng = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat,lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword="Mexican food", radius=5000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location=location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        # print(places_url, r.text)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def detail(self, place_id="ChIJlXOKcDC3j4ARzal-5j-p-FY", fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        detail_base_endpoint = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id": f"{place_id}",
            "fields" : ",".join(fields),
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
        

In [49]:
client = GoogleMapsClient(api_key=GOOGLE_API_KEY, address_or_postal_code='Dammam Saudi Arabia')
print(client.lat, client.lng)

26.4206828 50.0887943


In [50]:
client.search("burger", radius=5000)

{'html_attributions': [],
 'next_page_token': 'ARywPAIbNKQ7gGj7duGopOvdtbw8JsJ_qdzTnYQrMJIdQPsjpaKZ0lPIoVjssmbUAiQhttlucPkY6WxDgGHNf6h2CHfjitj4liB8q8qxdvVgnNSp0YF8Joq_CD3E3XlhXSzxrkL_e9DGlDNvfx1yrqa789HptUZvEAPnHEIKNqu9szCEPvtO9w8bT04MBG1FAooYRO9pbkd04Dn4XVwOsW3Xq2jbT0Gx0Ez7vlOHx8vwg1sIB94XaZ02vY9Pcr_ffdiWua1-MhKHOEndDGDRSPnzyb1250KPql5AVfakfJZOx3ICzfxzG_r0fgivs1orS0eyy67AyVbfvo7XD3wci6nYtl9ML-aq-GKUgWyAdy4ggu99pe9nB4uyUJs-qljW32_FSEj3m04OyeT9jPqHqf_AiWPiB14gr5nqjWDRVPCjPXpIvD3m--DfBIM',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 26.3882877, 'lng': 50.0750726},
    'viewport': {'northeast': {'lat': 26.38978397989272,
      'lng': 50.07663522989272},
     'southwest': {'lat': 26.38708432010727, 'lng': 50.07393557010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet

In [52]:
client.detail(place_id='ChIJ9WJVY-39ST4RLR-kZP59bpc')

{'html_attributions': [],
 'result': {'formatted_address': 'West, Avenue Mall, Omar Ibn Al Khattab St, Al Faysaliyah, Dammam 32271, Saudi Arabia',
  'formatted_phone_number': '9200 00002',
  'name': 'Burger King - West Avenue Mall',
  'rating': 4.3},
 'status': 'OK'}